# Inference notebook for Wealth Alpaca-LoRa
This notebook can be used to create a Gradio Interface that lets you interact with Wealth Alpaca-LoRa, an implementation of Alpaca-LoRa with a focus on wealth/finance. For training details on how you can replicate such a model using only free resources, checkout the repository: https://github.com/gaurangbharti1/wealth-alpaca

## Imports

In [1]:
!pip install datasets
!pip install sentencepiece
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install bitsandbytes
!pip install git+https://github.com/huggingface/peft.git
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-brdnuygx
  Running command git clone --filter=blob:none --quiet https://github.c

In [2]:
import torch
from peft import PeftModel
import transformers
import gradio as gr
assert (
    "LlamaTokenizer" in transformers._import_structure["models.llama"]
), "LLaMA is now in HuggingFace's main branch.\nPlease reinstall it: pip uninstall transformers && pip install git+https://github.com/huggingface/transformers.git"
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig

## Setup

In [3]:
tokenizer = LlamaTokenizer.from_pretrained("baffo32/decapoda-research-llama-7B-hf")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message


In [4]:
BASE_MODEL = "baffo32/decapoda-research-llama-7B-hf"
LORA_WEIGHTS = "kunchum/capstone-llama-finetuned"

In [5]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
try:
    if torch.backends.mps.is_available():
        device = "mps"
except:
    pass

In [6]:
if device == "cuda":
    model = LlamaForCausalLM.from_pretrained(
        BASE_MODEL,
        load_in_8bit=True,
        device_map="auto",
    )
    model = PeftModel.from_pretrained(
        model, LORA_WEIGHTS, force_download=True
    )

config.json:   0%|          | 0.00/428 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin.index.json:   0%|          | 0.00/25.5k [00:00<?, ?B/s]

pytorch_model-00001-of-00033.bin:   0%|          | 0.00/405M [00:00<?, ?B/s]

pytorch_model-00002-of-00033.bin:   0%|          | 0.00/405M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/399 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/399 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/peft/config.py:162: UserWarning: Unexpected keyword arguments ['enable_lora', 'merge_weights'] for class LoraConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(


adapter_model.bin:   0%|          | 0.00/8.41M [00:00<?, ?B/s]

In [7]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{instruction}
### Input:
{input}
### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
{instruction}
### Response:"""

In [8]:
model.half()
model.eval()
if torch.__version__ >= "2":
    model = torch.compile(model)

## Evaluate function setup

In [9]:
    generation_config = GenerationConfig(
        do_sample=True,
        temperature=0.6,
        top_p=0.95,
        repetition_penalty=1.15,
    )

In [10]:
def evaluate(
    instruction,
    input=None,
    temperature=0.1,
    top_p=0.75,
    top_k=40,
    num_beams=4,
    max_new_tokens=128,
    repetition_penalty=1.15,
    **kwargs,
):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    generation_config = GenerationConfig(
        temperature=temperature,
        do_sample=True,
        top_p=top_p,
        # top_k=top_k,
        # num_beams=num_beams,
        repetition_penalty=repetition_penalty,
        **kwargs,
    )
    with torch.autocast("cuda"):
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=max_new_tokens,
        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)
    return output.split("### Response:")[1].strip()

    #     response = ""
    # for s in generation_output.sequences:
    #     response = tokenizer.decode(generation_output.sequences[0], skip_special_tokens=True)

    # # Truncate the response at the first occurrence of triple quotes (""")
    # if '"""' in response:
    #     response = response.split('"""')[0].strip()
    #     response = response.split('### Response:')[1].strip()

    # return response

## Gradio Interface

In [11]:
import gradio as gr

g = gr.Interface(
    fn=evaluate,
    inputs=[
        gr.Textbox(
            lines=2, label="Instruction", placeholder="Tell me about the risks in money market accounts"
        ),
        gr.Textbox(lines=2, label="Input", placeholder="none"),
        gr.Slider(minimum=0, maximum=1, value=0.6, label="Temperature"),
        gr.Slider(minimum=0, maximum=1, value=0.95, label="Top p"),
        # gr.Slider(minimum=0, maximum=100, step=1, value=40, label="Top k"),
        # gr.Slider(minimum=1, maximum=4, step=1, value=4, label="Beams"),
        gr.Slider(minimum=1, maximum=512, step=1, value=128, label="Max tokens"),
        gr.Slider(minimum=0.1, maximum=2, step=0.1, value=1.15, label="Repetition Penalty"),
    ],
    outputs=[
        gr.Textbox(
            lines=10,
            label="Output",
        )
    ],
    title="FinAdvisor - Financial Advisory Tool for contextually-aware personal finance assistance",
    description="Wealth Alpaca-LoRA is a 7B-parameter LLaMA model finetuned to follow instructions focused on wealth/finance. It is trained on the [Stanford Alpaca](https://github.com/tatsu-lab/stanford_alpaca) dataset customized with FinQA and other data - more info here: https://huggingface.co/datasets/gbharti/finance-alpaca - and makes use of the Huggingface LLaMA implementation. For more information on Alpaca, please visit [Alpaca-LoRa](https://github.com/tloen/alpaca-lora) and for more info on this implementation, please visit [Wealth Alpaca-LoRa](https://github.com/gaurangbharti1/wealth-alpaca)",
)

# Remove concurrency_count argument
g.queue()
g.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://81a8847f0ef1532ebd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://81a8847f0ef1532ebd.gradio.live
